In [1]:
import cv2
import pytesseract
import matplotlib.pyplot as plt

In [ ]:
def function extract_text(image):
    # Load the image
    img = cv2.imread(image)

    # Convert the image to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Use pytesseract to extract text
    text = pytesseract.image_to_string(thresh)